In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based on this context {context}?
Question: {number} """

prompt = ChatPromptTemplate(template)

#Retriver is the context
#retriver = vectorstore.as_retriver(k=2)
retriver = ""

#llm is the llm to answer the q
llm = ""
rag_chain = {"context":retriver, "number":RunnablePassthrough()} | prompt | llm

question = "When did this happen?"
response = rag_chain.invoke(question)





c:\Users\masv\AppData\Local\miniforge3\envs\ai_agent_board_rules\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain.schema'

In [2]:
#Make conversational rag also. 

from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([HumanMessage(content=question)],
                    AIMessage(content=response))



NameError: name 'question' is not defined

In [3]:
from langchain_core.prompts import MessagesPlaceholder

sys_prompt = """
Given a chat history and a user question, answer the question with the previous context. 
"""

context_q_prompt = ChatPromptTemplate.from_messages([("system":sys_prompt),
                                                     (MessagesPlaceholder("chat_history")),
                                                     ("human":"{input}")])

contextualize_chain = context_q_prompt | llm

contextualize_chain.invoke({"input":"what is the thing?","chat_history":chat_history})


SyntaxError: invalid syntax (692873779.py, line 7)

In [2]:
from langchain_classic.chains import create_history_aware_retriever

history_aware_retriver = create_history_aware_retriever(llm,retriver,context_q_prompt)
history_aware_retriver.invoke({"input":"Whats the thing?","chat_history":chat_history})


c:\Users\masv\AppData\Local\miniforge3\envs\ai_agent_board_rules\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [ ]:
from langchain_classic.chains import create_retrival_chain 
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful chatbot"),
    ("system","Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
    ])

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrival_chain(history_aware_retriver,question_answer_chain)


#So this is final. What is happening? 

#The history_aware_retriver is essentially only getting relevant documents. These documents is based on the previous chat history.
#This here (the question_answer_chain) is just used for the last call to the llm. It takes the context (chunks from history) and the 
#chat_history (just so it understands what has been said) and the input from human. 
#Based on this it will answer the question. 


#
#rag_chain.invoke({"input":"Whats going on?","chat_history":chat_history})
